In [26]:
import pandas as pd
from openai import OpenAI
from ast import literal_eval
from embeddings import CHUNK_SIZE
from retrieval import query_embeddings


client = OpenAI()

In [27]:
df = pd.read_csv("data/20e01e08-12bd-4258-ab45-5cf9244b727f.csv")
df["embedding"] = df["embedding"].apply(literal_eval)
texts, embeddings = df["text"].tolist(), df["embedding"].tolist()

In [28]:
def query_results(query: str, debug=True) -> list[str]:
    results = query_embeddings(query, embeddings, 10)
    results_text = [texts[i] for i, _ in results]
    if debug:
        for i, result in enumerate(results_text):
            print(f"Result {i + 1} (Similarity: {results[i][1]}):")
            print(result)
            print("-" * 100)
    return results_text


def ask(query: str, results_text: list[str], debug=True):
    context = "\n\n###\n\n".join(results_text)
    system_message = "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"
    user_message = f"Context: {context}\n\n---\n\nQuestion: {query}\nAnswer:"

    if debug:
        print("System message:")
        print(system_message)
        print("User message:")
        print(user_message)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": user_message,
            },
        ],
    )

    return response.choices[0].message.content

In [29]:
query = "What is Stargate?"
results_text = query_results(query)

Result 1 (Similarity: 0.9342499421203767):
While some consumers and professionals have embraced ChatGPT and other conversational AI as well as AI-generated video, turning these recent breakthroughs into technology that produces significant revenue could take longer than practitioners in the field anticipated. Firms including Amazon and Google have quietly tempered expectations for sales, in part because such AI is costly and requires a lot of work to launch inside large enterprises or to power new features in apps used by millions of people. Altman said at an Intel event last month that AI models get “predictably better” when researchers throw more computing power at them. OpenAI has published research on this topic, which it refers to as the “scaling laws” of conversational AI. OpenAI “throwing ever more compute [power to scale up existing AI] risks leading to a ‘trough of disillusionment’” among customers as they realize the limits of the technology, said Ali Ghodsi, CEO of Databrick

In [30]:
ask(query, results_text)

System message:
Answer the question based on the context below, and if the question can't be answered based on the context, say "I don't know"


User message:
Context: While some consumers and professionals have embraced ChatGPT and other conversational AI as well as AI-generated video, turning these recent breakthroughs into technology that produces significant revenue could take longer than practitioners in the field anticipated. Firms including Amazon and Google have quietly tempered expectations for sales, in part because such AI is costly and requires a lot of work to launch inside large enterprises or to power new features in apps used by millions of people. Altman said at an Intel event last month that AI models get “predictably better” when researchers throw more computing power at them. OpenAI has published research on this topic, which it refers to as the “scaling laws” of conversational AI. OpenAI “throwing ever more compute [power to scale up existing AI] risks leading to a

"Stargate is a proposed data center project by Microsoft and OpenAI. It would contain a supercomputer with millions of specialized server chips to power OpenAI's artificial intelligence. The project is estimated to cost as much as $100 billion and is targeted to launch as soon as 2028."

In [31]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""Given a question, generate a paragraph of text that answers the question.
Question: {query}
Answer:
         """,
        },
    ],
    max_tokens=CHUNK_SIZE,
)

generated_text = response.choices[0].message.content
generated_text

'Stargate is a science fiction franchise that originated from the 1994 film of the same name, directed by Roland Emmerich. The core concept revolves around a device called the Stargate, which is an ancient, ring-shaped portal that allows for instant travel between different planets across the galaxy. The franchise expanded significantly with the launch of the television series "Stargate SG-1" in 1997, which followed a team of military personnel and scientists as they explored various worlds and encountered alien civilizations. This was followed by additional series such as "Stargate Atlantis" and "Stargate Universe," along with several television films and an animated series. The franchise explores themes of exploration, teamwork, and the moral complexities of interacting with other cultures, while also delving into mythology and the nature of technology. Stargate has garnered a dedicated fan base and is known for its creative storytelling and rich universe.'

In [32]:
generated_results_text = query_results(generated_text)

Result 1 (Similarity: 0.914370749553568):
An OpenAI spokesperson did not have a comment for this article. Altman has said privately that Google, one of OpenAI’s biggest rivals, will have more computing capacity than OpenAI in the near term, and publicly he has complained about not having as many AI server chips as he’d like. That’s one reason he has been pitching the idea of a new server chip company that would develop a chip rivaling Nvidia’s graphics processing unit, which today powers OpenAI’s software. Demand for Nvidia GPU servers has skyrocketed, driving up costs for customers such as Microsoft and OpenAI. Besides controlling costs, Microsoft has other potential reasons to support Altman’s alternative chip. The GPU boom has put Nvidia in the position of kingmaker as it decides which customers can have the most chips, and it has aided small cloud providers that compete with Microsoft. Nvidia has also muscled into reselling cloud servers to its own customers.
----------------------

In [33]:
ask(query, generated_results_text)

System message:
Answer the question based on the context below, and if the question can't be answered based on the context, say "I don't know"


User message:
Context: An OpenAI spokesperson did not have a comment for this article. Altman has said privately that Google, one of OpenAI’s biggest rivals, will have more computing capacity than OpenAI in the near term, and publicly he has complained about not having as many AI server chips as he’d like. That’s one reason he has been pitching the idea of a new server chip company that would develop a chip rivaling Nvidia’s graphics processing unit, which today powers OpenAI’s software. Demand for Nvidia GPU servers has skyrocketed, driving up costs for customers such as Microsoft and OpenAI. Besides controlling costs, Microsoft has other potential reasons to support Altman’s alternative chip. The GPU boom has put Nvidia in the position of kingmaker as it decides which customers can have the most chips, and it has aided small cloud providers 

'Stargate is the proposed U.S.-based supercomputer that Microsoft and OpenAI are planning to build. It is envisioned as the largest of a series of installations aimed at significantly increasing computing capacity for AI. The project could cost as much as $100 billion, according to initial cost estimates.'